In [ ]:
from pathlib import Path
import pandas as pd
import re
import os 
import shutil

# Copy Stuff

In [ ]:
correct_models_path = Path("correct_models/tensorflow/symbolic-cinit/16")
correct_models_path.mkdir(parents=True)

In [ ]:
mismatch_models_path = Path("mismatch_test_new/tensorflow/symbolic-cinit/16")
mismatch_models_path.mkdir(parents=True)

In [ ]:
# Copy Correct models
for idx, row in all_data.loc[(all_data['result'] == 0) & ~(all_data['mad'] > 1e-7)].iterrows():
    onnx_model_path = Path(row['onnx_model_path'])
    dest_path = correct_models_path / (str(Path(row['path']).parent.name) + "_repro")
    dest_path.mkdir(parents=True)
    dest_path = dest_path / "model.onnx"
    shutil.copy(onnx_model_path, dest_path)

In [ ]:
# Copy Incorrect models
for idx, row in all_data.loc[(all_data['result'] == 0) & (all_data['mad'] > 1e-7)].iterrows():
    tfnet_dest_path = Path(row['path']) / "tfnet"
    onnx_model_path = Path(row['onnx_model_path'])
    dest_path = mismatch_models_path / (str(Path(row['path']).parent.name) + "_repro")
    dest_path.mkdir(parents=True, exist_ok=True)
    dest_path_onnx = dest_path / "model.onnx"
    dest_path_tfnet = dest_path / "tfnet"
    shutil.copy(onnx_model_path, dest_path_onnx)
    shutil.copytree(tfnet_dest_path, dest_path_tfnet, dirs_exist_ok=True)


# Analysis

In [ ]:
results_path = Path("./exporter_test_results/tensorflow")


In [ ]:
dataframes = []
for path in sorted(results_path.glob(f"tensorflow_symbolic-cinit_opset_16_*.json")):
    dataframes.append(pd.read_json(path))

In [ ]:
all_data = pd.concat(dataframes).fillna(0)

In [ ]:
all_data.loc[all_data['mad'] > 1e-7].sort_values('mad', ascending=False).head(30)

In [ ]:
mismatch_models_path = Path("bugs/tensorflow/bug_1")
# Copy Incorrect models
for idx, row in all_data.loc[all_data['mad'] > 1e-7].sort_values('mad', ascending=False).head(30).iterrows():
    tfnet_dest_path = Path(row['path']) / "tfnet"
    onnx_model_path = Path(row['onnx_model_path'])
    dest_path = mismatch_models_path / (str(Path(row['path']).parent.name) + "_repro")
    dest_path.mkdir(parents=True, exist_ok=True)
    dest_path_onnx = dest_path / "model.onnx"
    dest_path_tfnet = dest_path / "tfnet"
    shutil.copy(onnx_model_path, dest_path_onnx)
    shutil.copytree(tfnet_dest_path, dest_path_tfnet, dirs_exist_ok=True)


In [ ]:
all_data[['result']].value_counts()

In [ ]:
all_data.loc[all_data['mad'] > 1e-7]

In [ ]:
num_ort = []
num_not_ort = []
n = 0
for idx, row in all_data.loc[all_data['result'] == 1].iterrows():
    if "ONNXRuntimeError" in row['error']:
        num_ort.append((row['model'], row['error']))
    else:
        num_not_ort.append((row['model'], row['error']))
        print(row)
    n += 1

In [ ]:
len(num_ort)

In [ ]:
x = list(set(list(map(lambda x: x[1], num_ort))))

In [ ]:
x

In [ ]:
fail = []
not_imp = []
invalid_arg = []
invalid_graph = []
runtime_exp = []
other = []
for i in x:
    if "NOT_IMPLEMENTED" in i:
        not_imp.append(i)
    elif "FAIL" in i:
        fail.append(i)
    elif "INVALID_ARGUMENT" in i:
        invalid_arg.append(i)
    elif "RUNTIME_EXCEPTION" in i:
        runtime_exp.append(i)
    elif "INVALID_GRAPH" in i:
        invalid_graph.append(i)
    else:
        other.append(i)


In [ ]:
print(f"{len(x)}, not_imp {len(not_imp)}, fail {len(fail)}, invalid arg {len(invalid_arg)}, invalid graph {len(invalid_graph)}, runtime_exp {len(runtime_exp)}" )

In [ ]:
fail

In [ ]:
runtime_exp

In [ ]:
len(num_not_ort)

In [ ]:
n

In [ ]:
x = list(set(list(map(lambda x: x[1], num_not_ort))))
x

In [ ]:
for i in all_data.loc[all_data['error'] == x[-1]]['path']:
    print(i)